In [1]:
# --- IMPORTAZIONI ---
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from numpy.fft import fft2, fftshift
import cv2
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as keras_image
import pandas as pd


2025-06-04 16:17:38.838354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# --- FUNZIONI UTILI ---
def load_and_preprocess_image(img_path):
    img = keras_image.load_img(img_path, target_size=(512, 512))
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def load_feature_extractor():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512, 512, 3))
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_conv3').output)
    return model

def extract_features(img, model):
    features = model.predict(img, verbose=0)
    return features.flatten()

def compute_metrics(original, cloaked):
    orig = ((original[0] + 1) / 2).clip(0, 1)
    cloak = ((cloaked[0] + 1) / 2).clip(0, 1)
    psnr_val = psnr(orig, cloak, data_range=1.0)
    ssim_val = ssim(orig, cloak, data_range=1.0, channel_axis=-1)
    return psnr_val, ssim_val

def show_images(img1, img2, title1="Originale", title2="Cloaked"):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(((img1[0] + 1) / 2).clip(0, 1))
    axs[0].set_title(title1)
    axs[0].axis('off')
    axs[1].imshow(((img2[0] + 1) / 2).clip(0, 1))
    axs[1].set_title(title2)
    axs[1].axis('off')
    plt.tight_layout()
    plt.show()

def show_difference_map(original, cloaked):
    diff = np.abs(original[0] - cloaked[0])
    diff /= diff.max()
    plt.imshow(diff)
    plt.title("Mappa delle differenze (assolute)")
    plt.axis('off')
    plt.colorbar()
    plt.show()

def show_fft(image, title):
    gray = cv2.cvtColor(((image[0] + 1) / 2).astype(np.float32), cv2.COLOR_RGB2GRAY)
    f = fft2(gray)
    fshift = fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)
    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.title(f"Spettro FFT - {title}")
    plt.axis('off')
    plt.show()


In [6]:
# --- CARICAMENTO IMMAGINI ---
# Modifica i path con quelli corretti nel tuo PC
original_path = "Pikachu.png"
#your_cloaked_path = "OutputLocker.png"   # <-- salva qui la tua immagine cloakata
glaze_low_path = "Pikachu-protected-intensity-LOW-V2.png"
glaze_default_path = "Pikachu-protected-intensity-DEFAULT-V2.png"
glaze_high_path = "Pikachu-protected-intensity-HIGH-V2.png"


# Carica immagini
original = load_and_preprocess_image(original_path)
#your_cloaked = load_and_preprocess_image(your_cloaked_path)
glaze_low = load_and_preprocess_image(glaze_low_path)
glaze_default = load_and_preprocess_image(glaze_default_path)
glaze_high = load_and_preprocess_image(glaze_high_path)

# Carica modello
model = load_feature_extractor()


2025-06-04 16:21:37.130306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# --- ANALISI E VISUALIZZAZIONI ---
results = []
images = [
    ("Tuo Cloaked", your_cloaked),
    ("Glaze LOW", glaze_low),
    ("Glaze DEFAULT", glaze_default),
    ("Glaze HIGH", glaze_high)
]

# Feature originale
features_original = extract_features(original, model)

for name, img in images:
    psnr_val, ssim_val = compute_metrics(original, img)
    features_img = extract_features(img, model)
    l2_distance = np.linalg.norm(features_original - features_img)

    results.append({
        "Nome": name,
        "PSNR (dB)": psnr_val,
        "SSIM": ssim_val,
        "L2 distance": l2_distance
    })

    print(f"🖼️ Analisi: {name}")
    show_images(original, img, "Originale", name)
    show_difference_map(original, img)
    show_fft(img, f"FFT - {name}")
    print("\n")


NameError: name 'your_cloaked' is not defined

In [ ]:
# --- TABELLA RIASSUNTIVA FINALE ---
df_results = pd.DataFrame(results)
df_results.set_index("Nome", inplace=True)
display(df_results)
